<a href="https://colab.research.google.com/github/sindhu819/Spark-2020/blob/main/Copy_of_Spark_RDD_pair_operation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **RDD pair Operation**

To run spark in Colab, we need to first install pyspark


In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,742 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main amd64 Package

In [ ]:
from pyspark import SparkContext
sc = spark.sparkContext

##Pair RDD:#
*	Key-value pair,(key, value), is a tuple with two elements
*	Keys and values can be atomic values, collections, or objects
*	Keys need not be unique among different pairs
*	Key-value semantics enables parallel computation
*	RDD partitioning: pairs with the same key can be grouped into one partition for efficient computation
*	RDD operations equally apply to pair RDDs
*	Allow aggregate, sort, and group by keys





In [ ]:
userVisits = [ ("chris","sjsu.edu"), ("jane","planetcassandra.org"),("chris","spark.apache.org") ]
pairRDD = sc.parallelize(userVisits)
print (pairRDD.collect())
# Let's keep only those with key as "chris"
chris_only = pairRDD.filter(lambda x:x[0] == "chris")
print(chris_only.collect())
chris_only_cap = chris_only.map(lambda x: (x[0].capitalize(), x[1]))
print (chris_only_cap.collect())
for i in range (0, chris_only_cap.count()):
  print ("%s visited the website www.%s" % (chris_only_cap.collect()[i][0],chris_only_cap.collect()[i][1]))

[('chris', 'sjsu.edu'), ('jane', 'planetcassandra.org'), ('chris', 'spark.apache.org')]
[('chris', 'sjsu.edu'), ('chris', 'spark.apache.org')]
[('Chris', 'sjsu.edu'), ('Chris', 'spark.apache.org')]
Chris visited the website www.sjsu.edu
Chris visited the website www.spark.apache.org


In [ ]:
pairRDD = sc.parallelize([("san",1),("jose",2),("san",3)])
print("Sort by keys in pairRDD: %s" % pairRDD.sortByKey(True).collect())
print("The keys in pairRDD: %s" % pairRDD.keys().distinct().collect())
print("The values in pairRDD: %s" % pairRDD.values().collect())
print("The values matching the key san:", pairRDD.lookup("san"))
print("key-value pairs (if a key is repeated, only the last one will be shown):", pairRDD.collectAsMap()) # if we use collect as map it returns the dict without repeats
print("Count by the key:", pairRDD.countByKey())

Sort by keys in pairRDD: [('jose', 2), ('san', 1), ('san', 3)]
The keys in pairRDD: ['san', 'jose']
The values in pairRDD: [1, 2, 3]
The values matching the key san: [1, 3]
key-value pairs (if a key is repeated, only the last one will be shown): {'san': 3, 'jose': 2}
Count by the key: defaultdict(<class 'int'>, {'san': 2, 'jose': 1})


In [ ]:
pairRDD = sc.parallelize([("san",1),("jose",2),("san",3)])
pairRDD.collectAsMap()

{'jose': 2, 'san': 3}

In [ ]:
pairRDD_san_1 = pairRDD.filter(lambda x: (x[0] == "san" and x[1] > 1))
print("Filtering those RDD with san in key and value great than 1:",pairRDD_san_1.collect())
print("Capitalize the first letter of the key and squarign the value: ",pairRDD_san_1.map( lambda t:(t[0].capitalize(), t[1] * t[1])).collect())

Filtering those RDD with san in key and value great than 1: [('san', 3)]
Capitalize the first letter of the key and squarign the value:  [('San', 9)]


In [ ]:
pairRDD_san_2 = pairRDD.filter(lambda x: (x[0] == "jose" and x[1] ==2))
print("Filtering those RDD with san in key and value great than 1:",pairRDD_san_2.collect())

Filtering those RDD with san in key and value great than 1: [('jose', 2)]


In [ ]:
pairRDD.collect()[1]

('jose', 2)

In [ ]:
print(sc.defaultParallelism)

2


### Inner Join ###

In [ ]:
rdd1 = sc.parallelize([ ("k1",1),("k5",5),("k3",3),("k3",33) ])
rdd2 = sc.parallelize([("k1",1),("k2",2),("k1",11),("k3",3) ])
rdd3 = rdd1.join(rdd2)
rdd3.collect()


[('k1', (1, 1)), ('k1', (1, 11)), ('k3', (3, 3)), ('k3', (33, 3))]

### Left Outer Join ###

In [ ]:
rdd1 = sc.parallelize([ ("k1",1),("k5",5),("k3",3),("k3",33) ])
rdd2 = sc.parallelize([("k1",1),("k2",2),("k1",11),("k3",3) ])
rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.collect()

[('k1', (1, 1)),
 ('k1', (1, 11)),
 ('k5', (5, None)),
 ('k3', (3, 3)),
 ('k3', (33, 3))]

### Right Outer Join ###

In [ ]:
rdd1 = sc.parallelize([ ("k1",1),("k5",5),("k3",3),("k3",33) ])
rdd2 = sc.parallelize([("k1",1),("k2",2),("k1",11),("k3",3) ])
rdd3 = rdd1.rightOuterJoin(rdd2)
rdd3.collect()

[('k1', (1, 1)),
 ('k1', (1, 11)),
 ('k2', (None, 2)),
 ('k3', (3, 3)),
 ('k3', (33, 3))]

### subtractByKey example ####

In [ ]:
rdd1.subtractByKey(rdd2).collect()

[('k5', 5)]

### customized parallelism  ###

In [ ]:
%%time
data = [("a", 3), ("b", 4), ("a", 1)]
standard = sc.parallelize(data).reduceByKey(lambda x, y: x + y) # Default parallelilism with numTask default value as 8
print(standard.collect())

[('b', 4), ('a', 4)]
CPU times: user 13.5 ms, sys: 2.47 ms, total: 16 ms
Wall time: 237 ms


In [ ]:
%%time
data = [("a", 3), ("b", 4), ("a", 1)]
customized = sc.parallelize(data).reduceByKey(lambda x, y: x + y, 10) # Custom parallelism with numTasks set to 10
print(customized.collect())

[('b', 4), ('a', 4)]
CPU times: user 14 ms, sys: 3.21 ms, total: 17.2 ms
Wall time: 562 ms


###We will import an csv file to illustrate some more RDD opearations.

In [ ]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1ilCXuYltyjv1auHn8FY_Csksep9gz03c'}) # replace the id with id of file you want to access
downloaded.GetContentFile('video-years.csv')

## Example 1 ##

In [ ]:
records = sc.textFile('video-years.csv')
records.collect() # See what got imported into this RDD

['b6ea95c5-9aea-11e5-a6ca-8b496c707234,"Monsters Inc.",2001',
 'b6d734d1-9aea-11e5-a6ca-8b496c707234,"Stuart Little 2",2002',
 'b6c5cfb5-9aea-11e5-a6ca-8b496c707234,"Piglet\'s Big Movie",2003',
 'b6d22bc1-9aea-11e5-a6ca-8b496c707234,"Jimmy Neutron: Boy Genius",2001',
 'b6c31093-9aea-11e5-a6ca-8b496c707234,"The Country Bears",2002',
 'b6d89469-9aea-11e5-a6ca-8b496c707234,"The Jungle Book 2",2003',
 'b6cd70d5-9aea-11e5-a6ca-8b496c707234,"The Tigger Movie",2000',
 'b6c31095-9aea-11e5-a6ca-8b496c707234,"Tarzan & Jane",2002',
 'b6d9cce1-9aea-11e5-a6ca-8b496c707234,"Finding Nemo",2003']

In [ ]:
records.count()

9

### Create key-value RDD pairs.###

In [ ]:
pair = records.map(lambda x: (x.split(",")[1].replace("\"",""),x.split(",")[2]))
pair.take(3)

[('Monsters Inc.', '2001'),
 ('Stuart Little 2', '2002'),
 ("Piglet's Big Movie", '2003')]

In [ ]:
type(pair.take(2))

list

In [ ]:
pair1 = records.map(lambda x: (x.split(",")[1],x.split(",")[2]))
pair1.take(3)

[('"Monsters Inc."', '2001'),
 ('"Stuart Little 2"', '2002'),
 ('"Piglet\'s Big Movie"', '2003')]